## icmecat

makes the interplanetary coronal mass ejection catalog ICMECATv2.0

Author: C. Möstl, IWF Graz, Austria
twitter @chrisoutofspace, part of https://github.com/cmoestl/heliocats

last update April 2020

Install a specific conda environment to run this code, see readme at https://github.com/cmoestl/heliocats

Adding a new ICME event: edit the file icmecat/HELCATS_ICMECAT_v20_master.xlsx to add 3 times, the id and spacecraft name, 
delete the file for the respective spacecraft under data/indices_icmecat, and run this notebook or script.

Convert this notebook to a script with jupyter nbconvert --to script icmecat.ipynb


**current status: work in progress**

features to be added: 
- finish new Wind data
- new events for Wind > 2018, new events STA > 2018
- new STA, Wind and PSP data magnetic field converted to SCEQ or HEEQ components 
- new STA data has beacon magnetic field for quick testing, replace with science data
- MAVEN CME verification with RAD and HI    


In [3]:
import numpy as np
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import  DateFormatter
import seaborn as sns
import datetime
import astropy
import astropy.constants as const
from sunpy.time import parse_time
import time
import pickle
import sys
import os
import urllib
import json
import importlib
import pandas as pd
import copy
import openpyxl
import h5py


from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging

from heliocats import data as hd
importlib.reload(hd) #reload again while debugging

from heliocats import cats as hc
importlib.reload(hc) #reload again while debugging

from heliocats import stats as hs
importlib.reload(hs) #reload again while debugging

#where the 5 in situ data files are located is read 
#from config.py 
from config import data_path


########### make directories first time if not there

resdir='results'
if os.path.isdir(resdir) == False: os.mkdir(resdir)

datadir='data'
if os.path.isdir(datadir) == False: os.mkdir(datadir)

indexdir='icmecat/indices_icmecat' 
if os.path.isdir(indexdir) == False: os.mkdir(indexdir) 


catdir='icmecat'
if os.path.isdir(catdir) == False: os.mkdir(catdir)

icplotsdir='icmecat/plots_icmecat/' 
if os.path.isdir(icplotsdir) == False: os.mkdir(icplotsdir) 

## (1) load new data with HelioSat and heliocats.data

In [4]:
load_data=1

if load_data > 0:

    print('load new Wind, STEREO-A, MAVEN, Parker Solar Probe data')

    # MAVEN
    #filemav='maven_2014_2018.p'
    #[mav,hmav]=pickle.load(open(filemav, 'rb' ) )

    #filemav='maven_2014_2018_removed.p'
    #[mav,hmav]=pickle.load(open(filemav, 'rb' ) )
    
    filemav='maven_2014_2018_removed_smoothed.p'
    [mav,hmav]=pickle.load(open(data_path+filemav, 'rb' ) )

    # Wind
    filewin="wind_2018_2019_heeq.p" 
    #filewin="wind_2018_2019_heeq.p" 
    #for updating data
    #start=datetime.datetime(2018, 1, 1)
    #end=datetime.datetime(2020, 1, 1)
    #hd.save_wind_data(data_path,filewin,start,end,heeq=True)
    [win2,hwin2]=pickle.load(open(data_path+filewin, "rb" ) )  

    # STEREO-A    
    #filesta2='stereoa_2015_2019_sceq.p'
    filesta2='stereoa_2015_2019.p'
    #start=datetime.datetime(2015, 7, 21)
    #end=datetime.datetime(2020, 1, 1)
    #hd.save_stereoa_science_data(data_path,filesta,start,end,sceq=True)   
    [sta2,hsta2]=pickle.load(open(data_path+filesta2, "rb" ) )  

    # Parker Solar Probe
    #filepsp='psp_2018_2019_sceq.p'
    filepsp='psp_2018_2019.p'
    #hd.save_psp_data(data_path,filepsp, sceq=True)   
    [psp,hpsp]=pickle.load(open(data_path+filepsp, "rb" ) )  


    # ADD BepiColombo  
    
    
    # ADD Solar Orbiter
    
    # get data file from helcats with headers
    [vex,win1,mes,sta1,stb,uly,hvex,hwin1,hmes,hsta1,hstb,huly]=hd.load_helcats_datacat(data_path+'helcats_all_data_removed.p') 
    print('done')
    
    #test HEEQ conversion for Wind and new Wind data in general
    #plt.plot_date(win1.time, win1.bz,'-k')
    #plt.plot_date(win2.time, win2.bz,'-g')
    #plt.xlim(parse_time('2018-1-20').plot_date,parse_time('2018-1-25').plot_date)
    #plt.plot_date(win1.time, win1.vt,'-g',linewidth=5)
    #plt.plot_date(win2.time, win2.vt,'-k')
    #plt.xlim(parse_time('2018-1-1').plot_date,parse_time('2018-1-3').plot_date)
    #plt.ylim((300,500))


    #########    
    print('merge old and new STEREO-A and Wind data')

    #merge STEREO-A old and new data    
    #make array
    sta=np.zeros(np.size(sta1.time)+np.size(sta2.time),dtype=[('time',object),('bx', float),('by', float),\
                ('bz', float),('bt', float),('vt', float),('np', float),('tp', float),\
                ('x', float),('y', float),('z', float),\
                ('r', float),('lat', float),('lon', float)])   

    #convert to recarray
    sta = sta.view(np.recarray)  
    #add merged variables
    sta.time=np.hstack((sta1.time,sta2.time))
    sta.bx=np.hstack((sta1.bx,sta2.bx))
    sta.by=np.hstack((sta1.by,sta2.by))
    sta.bz=np.hstack((sta1.bz,sta2.bz))
    sta.bt=np.hstack((sta1.bt,sta2.bt))
    sta.vt=np.hstack((sta1.vt,sta2.vt))
    sta.np=np.hstack((sta1.np,sta2.np))
    sta.tp=np.hstack((sta1.tp,sta2.tp))
    sta.x=np.hstack((sta1.x,sta2.x))
    sta.y=np.hstack((sta1.y,sta2.y))
    sta.z=np.hstack((sta1.z,sta2.z))
    sta.r=np.hstack((sta1.r,sta2.r))
    sta.lon=np.hstack((sta1.lon,sta2.lon))
    sta.lat=np.hstack((sta1.lat,sta2.lat))


    #merge Wind old and new data 
    #cut off HELCATS data at end of 2017, win2 begins exactly after this
    win1=win1[np.where(win1.time < parse_time('2018-Jan-01 00:00').datetime)[0]]
    #make array
    win=np.zeros(np.size(win1.time)+np.size(win2.time),dtype=[('time',object),('bx', float),('by', float),\
                ('bz', float),('bt', float),('vt', float),('np', float),('tp', float),\
                ('x', float),('y', float),('z', float),\
                ('r', float),('lat', float),('lon', float)])   

    #convert to recarray
    win = win.view(np.recarray)  
    win.time=np.hstack((win1.time,win2.time))
    win.bx=np.hstack((win1.bx,win2.bx))
    win.by=np.hstack((win1.by,win2.by))
    win.bz=np.hstack((win1.bz,win2.bz))
    win.bt=np.hstack((win1.bt,win2.bt))
    win.vt=np.hstack((win1.vt,win2.vt))
    win.np=np.hstack((win1.np,win2.np))
    win.tp=np.hstack((win1.tp,win2.tp))
    win.x=np.hstack((win1.x,win2.x))
    win.y=np.hstack((win1.y,win2.y))
    win.z=np.hstack((win1.z,win2.z))
    win.r=np.hstack((win1.r,win2.r))
    win.lon=np.hstack((win1.lon,win2.lon))
    win.lat=np.hstack((win1.lat,win2.lat))

    print('Merging done')
    print()

  
        
    
print()       
print('time ranges of the data: ')    
print('VEX ')
print(vex.time[0],vex.time[-1])
print('MESSENGER ')
print(mes.time[0],mes.time[-1])
print('Wind HELCATS data')
print(win1.time[0],win1.time[-1])
print('Wind new data')
print(win2.time[0],win2.time[-1])
print('STEREO-A HELCATS data ')
print(sta1.time[0],sta1.time[-1])      
print('STEREO-A new data')
print(sta2.time[0],sta2.time[-1])
print('STEREO-B ')
print(stb.time[0],stb.time[-1])
print('Ulysses ')
print(uly.time[0],uly.time[-1])
print('Parker Solar Probe ')
print(psp.time[0],psp.time[-1])

load new Wind, STEREO-A, MAVEN, Parker Solar Probe data
load all helcats DATACAT from single file:  /nas/helio/data/insitu_python/helcats_all_data_removed.p
Use vex,win,sta,stb,mes,uly to access data and position, hvex,hwin, hmes, hsta, hstb, huly for headers.
done
merge old and new STEREO-A and Wind data
Merging done


time ranges of the data: 
VEX 
2007-01-01 00:00:00 2014-11-25 23:58:00
MESSENGER 
2007-04-01 00:03:00 2015-04-30 15:08:00
Wind HELCATS data
2007-01-01 00:00:00 2017-12-31 23:59:00
Wind new data
2018-01-01 00:00:00 2019-12-31 23:58:00
STEREO-A HELCATS data 
2007-01-01 00:00:00 2015-03-20 16:01:00
STEREO-A new data
2015-07-21 00:00:00 2019-12-30 23:59:00
STEREO-B 
2007-01-01 00:00:00 2014-09-27 16:31:00
Ulysses 
1990-10-07 00:00:00 2009-12-31 22:00:00
Parker Solar Probe 
2018-10-14 14:15:00 2019-05-31 14:14:00


### 1a save data as numpy structured arrays for machine learning if needed

In [5]:
# save data as numpy structured arrays for machine learning
data_to_numpy=0

if data_to_numpy > 0:  
    print('convert data to numpy structured arrays suitable for machine learning')

    #STEREO-A
    #header with only vt total speed
    header_num1='dtype=[(time, matplotlib), (bt [nT], <f8), (bx  [nT, SCEQ], <f8), (by  [nT, SCEQ], <f8),\
                        (bz  [nT, SCEQ], <f8), (vt  [km/s], <f8), (np [ccm -3], <f8), (tp [K], <f8), \
                        (x [AU], <f8), (y [AU], <f8), (z [AU], <f8), (r [AU], <f8), (lat [deg], <f8), (lon [deg], <f8]'        
    sta_num=hd.recarray_to_numpy_array(sta)
    #save merged data extra
    pickle.dump([sta,header_num1], open(data_path+ "stereoa_2007_2019.p", "wb" ) )
    pickle.dump([sta_num,header_num1], open(data_path+ "stereoa_2007_2019_ndarray.p", "wb" ) )

    #Wind
    header_num2='dtype=[(time, matplotlib), (bt [nT], <f8), (bx  [nT, HEEQ], <f8), (by  [nT, HEEQ], <f8), \
                        (bz  [nT, HEEQ], <f8), (vt  [km/s], <f8), (np [ccm -3], <f8), \
                        (tp [K], <f8), (x [AU], <f8), (y [AU], <f8), (z [AU], <f8), (r [AU], <f8),\
                        (lat [deg], <f8), (lon [deg],<f8 )]'
    win_num=hd.recarray_to_numpy_array(win)
    pickle.dump([win,header_num2], open(data_path+ "wind_2007_2019.p", "wb" ) )
    pickle.dump([win_num,header_num2], open(data_path+ "wind_2007_2019_ndarray.p", "wb" ) )


    #Ulysses
    header_num3='dtype=[(time, matplotlib), (bt [nT], <f8), (bx  [nT, RTN], <f8), (by  [nT, RTN], <f8), \
                        (bz  [nT, RTN], <f8), (vt  [km/s], <f8), (np [ccm -3], <f8), (tp [K], <f8), (x [AU], <f8),\
                        (y [AU], <f8), (z [AU], <f8), (r [AU], <f8), (lat [deg], <f8), (lon [deg], <f8 )]'
    uly_num=hd.recarray_to_numpy_array(uly)
    #pickle.dump([uly,header_num1], open(data_path+ "ulysses_1990_2009_recarr.p", "wb" ) )
    pickle.dump([uly_num,header_num3], open(data_path+ "ulysses_1990_2009_ndarray.p", "wb" ) )


    #STEREO-B
    header_num4='dtype=[(time, matplotlib), (bt [nT], <f8), (bx, SCEQ  [nT], <f8), (by  [nT, SCEQ], <f8),\
                        (bz, SCEQ  [nT], <f8), (vt  [km/s], <f8),(vx  [km/s], <f8)(vy  [km/s], <f8),\
                        (vz  [km/s], <f8), (np [ccm -3], <f8), (tp [K], <f8), (x [AU], <f8), (y [AU], <f8),\
                        (z [AU], <f8), (r, <f8), (lat [deg], <f8), (lon [deg], <f8 )]'
    stb_num=hd.recarray_to_numpy_array(stb)
    #pickle.dump([stb,header_num2], open(data_path+ "stereob_2007_2014_recarr.p", "wb" ) )
    pickle.dump([stb_num,header_num4], open(data_path+ "stereob_2007_2014_ndarray.p", "wb" ) )

    #PSP    
    header_num5='dtype=[(time, matplotlib), (bt [nT], <f8), (bx, SCEQ  [nT], <f8), (by  [nT, SCEQ], <f8),\
                        (bz, RTN  [nT], <f8), (vt  [km/s], <f8),(vx  [km/s, RTN], <f8)(vy  [km/s, RTN], <f8),\
                        (vz  [km/s, RTN], <f8), (np [ccm -3], <f8), (tp [K], <f8), (x [AU], <f8), (y [AU], <f8),\
                        (z [AU], <f8), (r, <f8), (lat [deg], <f8), (lon [deg], <f8])'

    psp_num=hd.recarray_to_numpy_array(psp)
    pickle.dump([psp_num,header_num5], open(data_path+ "psp_2018_2019_ndarray.p", "wb" ) ) 

    #VEX, MESSENGER
    #header no plasma, with planetary orbit
    header_num6='dtype=[(time, matplotlib), (bt [nT], <f8), (bx  [nT, SCEQ], <f8), (by  [nT, SCEQ], <f8), \
                        (bz  [nT, SCEQ], <f8), (x [AU], <f8), (y [AU], <f8), (z [AU], <f8), (r [AU], <f8), \
                        (lat [deg], <f8), (lon [deg]), (xo [km], <f8), (yo [km], <f8), (zo [km], <f8), \
                        (ro [km], <f8), (lato [deg], <f8), (lono [deg], <f8)]'

    vex_num=hd.recarray_to_numpy_array(vex)
    pickle.dump([vex_num,header_num6], open(data_path+ "vex_2007_2014_ndarray.p", "wb" ) )

    mes_num=hd.recarray_to_numpy_array(mes)
    pickle.dump([mes_num,header_num6], open(data_path+ "mes_2007_2015_ndarray.p", "wb" ) ) 



    print('done')

## (2) measure new events 

In [6]:
#for measuring new events use this function from heliocats.plot 
#plt.close('all')
#works in jupyter notebooks

#works in scripts
#matplotlib.use('qt5agg')  
#plt.ion()


#hp.plot_insitu_measure(psp, '2018-Nov-10','2018-Nov-15', 'PSP', 'results/plots_icmecat/')
#%matplotlib 



#Wind
#hp.plot_insitu_measure(win, '2019-Jan-29','2019-Feb-28', 'Wind', 'results/')


#STEREO-A
#hp.plot_insitu_measure(sta, '2018-Jan-01 12:00','2018-Feb-01 12:00', 'STEREO-A', 'results/')



#for plotting single events
#hp.plot_insitu(psp, ic.icme,'2018-Nov-15', 'PSP', icplotsdir)

#-----------------------

#read HIGEOCAT from https://www.helcats-fp7.eu/catalogues/wp3_cat.html
#https://docs.astropy.org/en/stable/io/votable/
from astropy.io.votable import parse_single_table
table = parse_single_table('data/HCME_WP3_V06.vot')
data = table.array
a=table.array['HM HEEQ Long'][10]
print(a)



#    "columns" : [ "ID", "Date [UTC]", "SC", "L-N", "PA-N [deg]", "L-S", "PA-S [deg]", "Quality" 
#       , "PA-fit [deg]"
#       , "FP Speed [kms-1]", "FP Speed Err [kms-1]", "FP Phi [deg]", "FP Phi Err [deg]","FP HEEQ Long [deg]",  "FP HEEQ Lat [deg]",  "FP Carr Long [deg]", "FP Launch [UTC]"
#       , "SSE Speed [kms-1]", "SSE Speed Err [kms-1]", "SSE Phi [deg]", "SSE Phi Err [deg]", "SSE HEEQ Long [deg]", "SSE HEEQ Lat [deg]",  "SSE Carr Long [deg]","SSE Launch [UTC]"
#       , "HM Speed [kms-1]", "HM Speed Err [kms-1]", "HM Phi [deg]", "HM Phi Err [deg]", "HM HEEQ Long [deg]", "HM HEEQ Lat [deg]", "HM Carr Long [deg]", "HM Launch [UTC]"
#  ],


111


## (3) make ICMECAT 

In [7]:
print('data loaded')
ic=hc.load_helcats_icmecat_master_from_excel('icmecat/HELCATS_ICMECAT_v20_master.xlsx')

####### 3a get indices for all spacecraft

wini=np.where(ic.sc_insitu == 'Wind')[:][0] 
stai=np.where(ic.sc_insitu == 'STEREO-A')[:][0]    
stbi=np.where(ic.sc_insitu == 'STEREO-B')[:][0]    
vexi=np.where(ic.sc_insitu == 'VEX')[:][0]  
mesi=np.where(ic.sc_insitu == 'MESSENGER')[:][0]   
ulyi=np.where(ic.sc_insitu == 'ULYSSES')[:][0]    
mavi=np.where(ic.sc_insitu == 'MAVEN')[:][0]    
pspi=np.where(ic.sc_insitu == 'PSP')[:][0]    

####### 3b get parameters for all spacecraft one after another

ic=hc.get_cat_parameters(win,wini,ic,'Wind')
ic=hc.get_cat_parameters(sta,stai,ic,'STEREO-A')
ic=hc.get_cat_parameters(stb,stbi,ic,'STEREO-B')
ic=hc.get_cat_parameters(vex,vexi,ic,'VEX')
ic=hc.get_cat_parameters(mes,mesi,ic,'MESSENGER')
ic=hc.get_cat_parameters(uly,ulyi,ic,'ULYSSES')
ic=hc.get_cat_parameters(mav,mavi,ic,'MAVEN')
ic=hc.get_cat_parameters(psp,pspi,ic,'PSP')


####### 3c make all plots if wanted
#matplotlib.use('Agg')
# hp.plot_icmecat_events(win,wini,ic,'Wind',icplotsdir)
# hp.plot_icmecat_events(sta,stai,ic,'STEREO-A',icplotsdir)
# hp.plot_icmecat_events(stb,stbi,ic,'STEREO-B',icplotsdir)
# hp.plot_icmecat_events(vex,vexi,ic,'VEX',icplotsdir)
# hp.plot_icmecat_events(mes,mesi,ic,'MESSENGER',icplotsdir)
# hp.plot_icmecat_events(uly,ulyi,ic,'ULYSSES',icplotsdir)
# hp.plot_icmecat_events(mav,mavi,ic,'MAVEN',icplotsdir)
# hp.plot_icmecat_events(psp,pspi,ic,'PSP',icplotsdir)


data loaded
load HELCATS ICMECAT from file: icmecat/HELCATS_ICMECAT_v20_master.xlsx
Get parameters for  Wind


/home/cmoestl/pycode/heliocats/heliocats/cats.py:234: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'sheath_speed_mean']=np.round(np.nanmean(sc.vt[icme_start_ind[i]:mo_start_ind[i]]),1)
/home/cmoestl/miniconda/envs/helio/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:237: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'sheath_density_mean']=np.round(np.nanmean(sc.np[icme_start_ind[i]:mo_start_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:242: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'sheath_pdyn_mean']=np.round(np.nanmean(pdyn_i),1)


Get parameters for  STEREO-A


/home/cmoestl/pycode/heliocats/heliocats/cats.py:170: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'icme_speed_mean']=np.round(np.nanmean(sc.vt[icme_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:216: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_speed_mean']=np.round(np.nanmean(sc.vt[mo_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:221: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_density_mean']=np.round(np.nanmean(sc.np[mo_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:224: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_temperature_mean']=np.round(np.nanmean(sc.tp[mo_start_ind[i]:mo_end_ind[i]]),1)
/home/cmoestl/pycode/heliocats/heliocats/cats.py:229: RuntimeWarning: Mean of empty slice
  ic.at[sci[i],'mo_pdyn_mean']=np.round(np.nanmean(pdyn_i),1)


Get parameters for  STEREO-B
Get parameters for  VEX
Get parameters for  MESSENGER
Get parameters for  ULYSSES
Get parameters for  MAVEN
Get parameters for  PSP


### (4) save ICMECAT 

### 4a save header

In [8]:
#save header and parameters as text file and prepare for html website
print('header:')
header='ICME CATALOGUE v2.0 \n\n\
This is the HELCATS interplanetary coronal mass ejection (ICME) catalog, based on in situ magnetic field and bulk plasma observations in the heliosphere. \n\n\
This is version 2.0, released in April 2020 with a major update to the original version 1.0, originally a product of EU HELCATS project (2014-2017). \n\n\
Released 2020-**-**. DOI: 10.6084/m9.figshare.4588315.v2 \n\
The catalog is available as .p (python pickle), .xlsx, .csv, .html, .json, hdf5 at https://helioforecast.space/icmecat \n\n\
Number of events in ICMECAT: '+str(len(ic))+' \n\
ICME observatories: Wind, STEREO-A, STEREO-B, Venus Express (VEX), MESSENGER, Ulysses, MAVEN, Parker Solar Probe (PSP).   \n\
Time range: January 2007 - December 2018. \n \n\
Authors: Christian Moestl, Andreas Weiss, Space Research Institute, Austrian Academy of Sciences, Graz, Austria.\n\
Contributors: Peter Boakes, Alexey Isavnin, Emilia Kilpua, Reka Winslow, Brian Anderson, Lydia Philpott, \
Vratislav Krupar, Jonathan Eastwood, Simon Good, Lan Jian, Teresa Nieves-Chinchilla, Cyril Simon Wedlund, Jingnan Guo, Mateja Dumbovic, Benoit Lavraud.  \n\n\
This catalog has been made by getting the 3 times of each ICME (shock or disturbance begin, \
magnetic obstacle start and end) from the individual catalogs below, and then calculating all parameters again consistently from the data by us. \n\
Each icmecat_id has a tag in it that indicates from which catalog the ICME times were taken. \n\n\
Wind:       Nieves-Chinchilla et al. (2018), tag: NASA. \n\
STEREO-A:   Jian et al. (2018), tag: JIAN. \n\
STEREO-B:   Jian et al. (2018), tag: JIAN. \n\
VEX:        Good et al. (2018), tag: SGOOD \n\
MESSENGER:  Good et al. (2018), Winslow et al. (2018), tags: SGOOD, WINSLOW. \n\
MAVEN:      Möstl et al. (2020, in prep.), tag: MOESTL.\n\
Ulysses:    Added by us, tag: MOESTL. \n\
PSP:        Added by us, tag: MOESTL. \n\
We have also added extra events at Ulysses, STEREO-A, PSP, MESSENGER, VEX, Wind and MAVEN (all tagged with MOESTL in icmecat_id).\n\n\
The in situ data (about 4 GB) can be downloaded in python pickle format as recarrays from https://doi.org/10.6084/m9.figshare.11973693 \n\
The python code for producing this catalog is available as a jupyter notebook and .py script: https://github.com/cmoestl/icmecat.ipynb \n\n\
References: \n\
Nieves-Chinchilla, T. et al. (2018), https://doi.org/10.1007/s11207-018-1247-z  https://wind.nasa.gov/fullcatalogue.php \n\
Jian, L. et al. (2018), https://doi.org/10.3847/1538-4357/aab189 https://stereo-ssc.nascom.nasa.gov/data/ins_data/impact/level3/ \n\
Good, S. et al. (2018) https://doi.org/10.1007/s11207-015-0828-3 \n\
Winslow, R. et al. (2015), https://doi.org/10.1002/2015JA021200 \n\
Möstl, C. et al. (2020) in preparation \n\n\n\
Comments: \n\
- Spacecraft positions are given in Heliocentric Earth Equatorial Coordinates (HEEQ) coordinates. \n\
- Coordinate system for all magnetic field components is SCEQ except Ulysses and Parker Solar Probe (RTN), MAVEN (MSO) and Wind (HEEQ). \n\
        Definition of SpaceCraft Equatorial Coordinates (SCEQ): \n\
        Z is the solar rotation axis. \n\
        X points from the Sun to the spacecraft, projected in the solar equatorial plane. \n\
        Y completes the right handed triad and points to solar west. \n\
        This system is thus centered on the respective in situ spacecraft. \n\
        The solar equatorial plane as the reference plane is similar for all spacecraft.\n\
- Venus Express and MESSENGER do not have plasma parameters available. \n\
- If there is no sheath region, so the ICME starts immediately with a magnetic obstacle, the icme_start_time is similar to mo_start_time.\n\
- At MESSENGER and VEX, for events cataloged by Simon Good, icme_start_time has been added by V. Krupar (Imperial College) and C. Möstl (IWF Graz). \n\
- For the calculation of the parameters at MESSENGER during the orbit around Mercury, all data points inside the bowshock of Mercury have been removed, \
according to a list thankfully provided to us by by R. Winslow, UNH, B. Anderson, APL, and Lydia Philpott, UBC. \n\
- Calculation of the magnetic obstacle parameters at VEX is done after approximate removal of the induced magnetosphere, with a modified equation \n\
in Zhang et al. 2008 (doi: 10.1016/j.pss.2007.09.012), with a constant of 3.5 instead of 2.14/2.364,\
in order to account for a larger bowshock distance during solar maximum than studied in this paper. \n\
- For MAVEN, all data inside the bow shock were removed with the model from ****************** (by C. Simon Wedlund). \
From the remaining data, the median for each orbit is taken as 1 data point, resulting in a solar wind dataset at Mars with 4.5 hour time resolution.\n\
- The identification of ICMEs for MAVEN is a mixture of methods using data from MSL/RAD, MAVEN and STEREO/HI (see Möstl et al. 2020, in prep.). \n\n\n\n'



parameters='Parameters:\n\
00: icmecat_id: The unique identifier for the observed ICME. unit: string. \n\
01: sc insitu: The name of the in situ observing spacecraft. unit: string. \n\
02: icme_start_time: Shock arrival or density enhancement time, can be similar to mo_start_time. unit: UTC. \n\
03: mo_start_time: Start time of the magnetic obstacle (MO), including flux ropes, flux-rope-like, and ejecta signatures. unit: UTC. \n\
04: mo_end_time: End time of the magnetic obstacle. unit: UTC. \n\
05: mo_sc_heliodistance: Heliocentric distance of the spacecraft at mo_start_time. unit: AU.\n\
06: mo_sc_long_heeq: Heliospheric longitude of the spacecraft at mo_start_time, range [-180,180]. unit: degree (HEEQ).\n\
07: mo_sc_lat_heeq: Heliospheric latitude of the spacecraft at mo_start_time, range [-90,90]. unit: degree (HEEQ).\n\
08: icme_duration: Duration of the interval between icme_start_time and mo_endtime. unit: hours.\n\
09: icme_bmax: Maximum total magnetic field in the full icme interval (icme_start_time to mo_end_time). unit: nT.\n\
10: icme_bmean: Mean total magnetic field during the full icme interval (icme_start_time to mo_end_time). unit: nT.\n\
11: icme_bstd: Standard deviation of the total magnetic field from icme_start_time to mo_end_time. unit: nT.\n\
12: icme_speed_mean: Mean proton speed from icme_start_time to mo_end_time. unit: km/s.\n\
13: icme_speed_std: Standard deviation of proton speed from icme_start_time to mo_end_time. unit: km/s.\n\
14: mo_duration: Duration of the interval between mo_start_time and mo_endtime. unit: hours.\n\
15: mo_bmax: Maximum total magnetic field in the magnetic obstacle interval (mo_start_time to mo_end_time). unit: nT.\n\
16: mo_bmean: Mean total magnetic field in the magnetic obstacle. unit: nT.\n\
17: mo_bstd: Standard deviation of the total magnetic field in the magnetic obstacle. unit: nT.\n\
18: mo_bzmean: Mean magnetic field Bz component in the magnetic obstacle. unit: nT.\n\
19: mo_bzmin: Minimum magnetic field Bz component in the magnetic obstacle. unit: nT.\n\
20: mo_bzstd: Standard deviation of the magnetic field Bz component in the magnetic obstacle. unit: nT.\n\
21: mo_bymean: Mean magnetic field By component in the magnetic obstacle. unit: nT.\n\
22: mo_bystd: Standard deviation of the magnetic field By component in the magnetic obstacle. unit: nT.\n\
23: mo_speed_mean: Mean proton speed from mo_start_time to mo_end_time. unit: km/s.\n\
24: mo_speed_std: Standard deviation of proton speed from mo_start_time to mo_end_time. unit: km/s.\n\
25: mo_expansion_speed: Difference between proton speed at mo_start_time to proton speed at mo_end_time. unit: km/s.\n\
26: mo_pdyn_mean: Mean proton dynamic pressure from mo_start_time to mo_start_time. unit: nPa.\n\
27: mo_pdyn_std: Standard deviation of proton dynamic pressure from mo_start_time to mo_start_time. unit: nPa.\n\
28: mo_density_mean: Mean proton density from mo_start_time to mo_start_time. unit: cm^-3.\n\
29: mo_density_std: Standard deviation of proton density from mo_start_time to mo_start_time. unit: cm^-3.\n\
30: mo_temperature_mean: Mean proton temperature from mo_start_time to mo_start_time. unit: K.\n\
31: mo_temperature_std: Standard deviation of proton temperature from mo_start_time to mo_end_time. unit: K.\n\
32: sheath_speed_mean: Mean proton speed from icme_start_time to mo_start_time, NaN if these times are similar. unit: km/s.\n\
33: sheath_speed_std: Standard deviation of proton speed from icme_start_time to mo_start_time, NaN if these times are similar. unit: km/s.\n\
34: sheath_density_mean: Mean proton density from icme_start_time to mo_start_time, NaN if these times are similar. unit: cm^-3.\n\
35: sheath_density_std: Standard deviation of proton density from icme_start_time to mo_start_time, NaN if these times are similar. unit: cm^-3.\n\
36: sheath_pdyn_mean: Mean proton dynamic pressure, from icme_start_time to mo_start_time, NaN if these times are similar. unit: nPa.\n\
37: sheath_pdyn_std: Standard deviation of proton dynamic pressure, from icme_start_time to mo_start_time, NaN if these times are similar. unit: nPa.\n\n\n'


print('-------------------------')
print(header)
print(parameters)
print('-------------------------')


#make header file
file='icmecat/HELCATS_ICMECAT_v20_header.txt'
with open(file, "w") as text_file:
    text_file.write(header)
    text_file.write(parameters)
print()    
print('header saved as '+file)
print()    

#Convert to html regarding line breaks, paragraph beginning and spaces
header_spaces=header.replace(" ", "&nbsp;")
header_html= "<p>" +header_spaces.replace('\n', '<br>')+ "</p>" 
parameters_spaces=parameters.replace(" ", "&nbsp;")
parameters_html= "<p>" +parameters.replace('\n', '<br>')+ "</p>"
print('header converted to HTML')
print()    
print()    

header:
-------------------------
ICME CATALOGUE v2.0 

This is the HELCATS interplanetary coronal mass ejection (ICME) catalog, based on in situ magnetic field and bulk plasma observations in the heliosphere. 

This is version 2.0, released in April 2020 with a major update to the original version 1.0, originally a product of EU HELCATS project (2014-2017). 

Released 2020-**-**. DOI: 10.6084/m9.figshare.4588315.v2 
The catalog is available as .p (python pickle), .xlsx, .csv, .html, .json, hdf5 at https://helioforecast.space/icmecat 

Number of events in ICMECAT: 698 
ICME observatories: Wind, STEREO-A, STEREO-B, Venus Express (VEX), MESSENGER, Ulysses, MAVEN, Parker Solar Probe (PSP).   
Time range: January 2007 - December 2018. 
 
Authors: Christian Moestl, Andreas Weiss, Space Research Institute, Austrian Academy of Sciences, Graz, Austria.
Contributors: Peter Boakes, Alexey Isavnin, Emilia Kilpua, Reka Winslow, Brian Anderson, Lydia Philpott, Vratislav Krupar, Jonathan Eastwood, S

### 4b save into different formats

In [9]:
########## python formats

# save ICMECAT as pandas dataframe with times as datetime objects as pickle
file='icmecat/HELCATS_ICMECAT_v20_pandas.p'
pickle.dump([ic,header,parameters], open(file, 'wb'))
print('ICMECAT saved as '+file)


# save ICMECAT as numpy array with times as matplotlib datetime as pickle
ic_num=copy.deepcopy(ic) 
ic_num.icme_start_time=parse_time(ic_num.icme_start_time).plot_date
ic_num.mo_start_time=parse_time(ic_num.mo_start_time).plot_date
ic_num.mo_end_time=parse_time(ic_num.mo_end_time).plot_date
#convert to recarray
ic_num_rec=ic_num.to_records()
#create structured array
dtype1=[('index','i8'),('icmecat_id', '<U30'),('sc_insitu', '<U20')] +[(i, '<f8') for i in ic.keys()[2:len(ic.keys())]]
ic_num_struct=np.array(ic_num_rec,dtype=dtype1)


file='icmecat/HELCATS_ICMECAT_v20_numpy.p'
pickle.dump([ic_num,ic_num_struct,header,parameters], open(file, 'wb'))
print('ICMECAT saved as '+file)



################ save to different formats

#copy pandas dataframe first to change time format consistent with HELCATS
ic_copy=copy.deepcopy(ic)  
ic_copy.icme_start_time=parse_time(ic.icme_start_time).isot
ic_copy.mo_start_time=parse_time(ic.mo_start_time).isot
ic_copy.mo_end_time=parse_time(ic.mo_end_time).isot

#change time format
for i in np.arange(len(ic)):

    dum=ic_copy.icme_start_time[i] 
    ic_copy.at[i,'icme_start_time']=dum[0:16]+'Z'
     
    dum=ic_copy.mo_start_time[i] 
    ic_copy.at[i,'mo_start_time']=dum[0:16]+'Z'
     
    dum=ic_copy.mo_end_time[i] 
    ic_copy.at[i,'mo_end_time']=dum[0:16]+'Z'


#save as Excel
file='icmecat/HELCATS_ICMECAT_v20.xlsx'
ic_copy.to_excel(file,sheet_name='ICMECATv2.0')
print('ICMECAT saved as '+file)

#save as json
file='icmecat/HELCATS_ICMECAT_v20.json'
ic_copy.to_json(file)
print('ICMECAT saved as '+file)

#save as csv
file='icmecat/HELCATS_ICMECAT_v20.csv'
ic_copy.to_csv(file)
print('ICMECAT saved as '+file)


#save as txt
file='icmecat/HELCATS_ICMECAT_v20.txt'
np.savetxt(file, ic_copy.values.astype(str), fmt='%s' )
print('ICMECAT saved as '+file)


#########################


#########save into hdf5 format , use S for strings http://docs.h5py.org/en/stable/strings.html#what-about-numpy-s-u-type
dtype2=[('index','i8'),('icmecat_id', 'S30'),('sc_insitu', 'S20')] +[(i, '<f8') for i in ic.keys()[2:len(ic.keys())]]
ich5=np.array(ic_num_rec,dtype=dtype2)
file='icmecat/HELCATS_ICMECAT_v20.h5'
f=h5py.File(file,mode='w')
f["icmecat"]= ich5
#add attributes
#************************
#***********************

print('ICMECAT saved as '+file)
f.close()

#reading h5py files http://docs.h5py.org/en/latest/quick.html
fr = h5py.File('icmecat/HELCATS_ICMECAT_v20.h5', 'r')
list(fr.keys())
ich5=fr['icmecat']
ich5['mo_bstd']
ich5.dtype
fr.close()
##################









############ other formats

#copy pandas dataframe first to change time format 
ic_copy2=copy.deepcopy(ic)  
ic_copy2.icme_start_time=parse_time(ic.icme_start_time).iso
ic_copy2.mo_start_time=parse_time(ic.mo_start_time).iso
ic_copy2.mo_end_time=parse_time(ic.mo_end_time).iso

#change time format
for i in np.arange(len(ic)):

    dum=ic_copy2.icme_start_time[i] 
    ic_copy2.at[i,'icme_start_time']=dum[0:16]
     
    dum=ic_copy2.mo_start_time[i] 
    ic_copy2.at[i,'mo_start_time']=dum[0:16]
     
    dum=ic_copy2.mo_end_time[i] 
    ic_copy2.at[i,'mo_end_time']=dum[0:16]


#save as json for webpage with different time format
file='icmecat/HELCATS_ICMECAT_v20_isot.json'
ic_copy2.to_json(file)
print('ICMECAT saved as '+file)


#save as html no header
file='icmecat/HELCATS_ICMECAT_v20_simple.html'
ic_copy.to_html(file)
print('ICMECAT saved as '+file)


############ save as html file with header
#save as html
file='icmecat/HELCATS_ICMECAT_v20.html'
#ic.to_html(file,justify='center')

#ichtml='{% extends "_base.html" %} \n \n {% block content %} \n \n \n '
ichtml = header_html
ichtml += parameters_html
ichtml += ic_copy.to_html()
#ichtml +='\n \n {% endblock %}'


with open(file,'w') as f:
    f.write(ichtml)
    f.close()
    
print('ICMECAT saved as '+file)    

ICMECAT saved as icmecat/HELCATS_ICMECAT_v20_pandas.p
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20_numpy.p
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.xlsx
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.json
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.csv
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.txt
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.h5
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20_isot.json
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20_simple.html
ICMECAT saved as icmecat/HELCATS_ICMECAT_v20.html


## 4c load ICMECAT pickle files

In [10]:
#load icmecat as pandas dataframe
file='icmecat/HELCATS_ICMECAT_v20_pandas.p'
[ic_pandas,h,p]=pickle.load( open(file, 'rb'))   

#load icmecat as numpy array
file='icmecat/HELCATS_ICMECAT_v20_numpy.p'
[ic_nprec,ic_np,h,p]=pickle.load( open(file, 'rb'))   


In [11]:
ic_pandas

,icmecat_id,sc_insitu,icme_start_time,mo_start_time,mo_end_time,mo_sc_heliodistance,mo_sc_long_heeq,mo_sc_lat_heeq,icme_duration,icme_bmax,...,mo_density_mean,mo_density_std,mo_temperature_mean,mo_temperature_std,sheath_speed_mean,sheath_speed_std,sheath_density_mean,sheath_density_std,sheath_pdyn_mean,sheath_pdyn_std
0,ICME_Wind_NASA_20070114_01,Wind,2007-01-14 11:31:00,2007-01-14 11:44:00,2007-01-15 07:45:00,0.9729,0.14,-4.43,20.23,14.6,...,5.0,2.1,15652.6,9616.6,310.4,1.2,1.1,0.1,0.2,0.0
1,ICME_Wind_NASA_20070115_01,Wind,2007-01-15 20:49:00,2007-01-15 20:49:00,2007-01-16 04:45:00,0.9731,0.15,-4.57,7.93,10.4,...,4.1,1.4,97358.1,37602.1,NaN,NaN,NaN,NaN,NaN,NaN
2,ICME_Wind_NASA_20070329_01,Wind,2007-03-29 15:04:00,2007-03-29 15:04:00,2007-03-30 00:03:00,0.9900,-0.02,-6.74,8.98,6.1,...,5.2,1.0,16707.4,8263.1,NaN,NaN,NaN,NaN,NaN,NaN
3,ICME_Wind_NASA_20070521_01,Wind,2007-05-21 22:40:00,2007-05-21 22:45:00,2007-05-22 13:25:00,1.0020,-0.23,-1.91,14.75,14.0,...,6.7,2.6,41975.6,17102.7,462.5,2.1,1.9,0.2,0.7,0.1
4,ICME_Wind_NASA_20070608_01,Wind,2007-06-08 05:45:00,2007-06-08 05:45:00,2007-06-09 05:15:00,1.0040,-0.13,0.19,23.50,9.8,...,7.3,1.9,53830.2,25294.9,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,ICME_MAVEN_MOESTL_20170119_01,MAVEN,2017-01-19 05:50:00,2017-01-19 15:00:00,2017-01-20 18:30:00,1.4262,-91.86,4.74,36.67,5.5,...,2.6,0.8,287437.6,242820.6,357.1,9.7,2.8,0.7,0.6,0.2
694,ICME_MAVEN_MOESTL_20170703_01,MAVEN,2017-07-03 22:20:00,2017-07-04 21:00:00,2017-07-06 04:40:00,1.6225,-168.71,-2.80,54.33,7.8,...,4.8,1.4,369548.7,456549.3,378.7,6.1,2.6,1.2,0.6,0.3
695,ICME_MAVEN_MOESTL_20170722_01,MAVEN,2017-07-22 14:30:00,2017-07-22 14:30:00,2017-07-24 11:50:00,1.6368,-177.51,-3.47,45.33,6.7,...,4.8,1.4,178292.6,220041.6,NaN,NaN,NaN,NaN,NaN,NaN
696,ICME_MAVEN_MOESTL_20170726_01,MAVEN,2017-07-26 04:30:00,2017-07-26 09:00:00,2017-07-27 12:10:00,1.6396,-179.41,-3.60,31.67,21.4,...,2.4,0.8,1167580.7,1128618.7,666.2,0.0,0.5,0.0,0.4,0.0


In [12]:
ic_nprec

,icmecat_id,sc_insitu,icme_start_time,mo_start_time,mo_end_time,mo_sc_heliodistance,mo_sc_long_heeq,mo_sc_lat_heeq,icme_duration,icme_bmax,...,mo_density_mean,mo_density_std,mo_temperature_mean,mo_temperature_std,sheath_speed_mean,sheath_speed_std,sheath_density_mean,sheath_density_std,sheath_pdyn_mean,sheath_pdyn_std
0,ICME_Wind_NASA_20070114_01,Wind,732690.479861,732690.488889,732691.322917,0.9729,0.14,-4.43,20.23,14.6,...,5.0,2.1,15652.6,9616.6,310.4,1.2,1.1,0.1,0.2,0.0
1,ICME_Wind_NASA_20070115_01,Wind,732691.867361,732691.867361,732692.197917,0.9731,0.15,-4.57,7.93,10.4,...,4.1,1.4,97358.1,37602.1,NaN,NaN,NaN,NaN,NaN,NaN
2,ICME_Wind_NASA_20070329_01,Wind,732764.627778,732764.627778,732765.002083,0.9900,-0.02,-6.74,8.98,6.1,...,5.2,1.0,16707.4,8263.1,NaN,NaN,NaN,NaN,NaN,NaN
3,ICME_Wind_NASA_20070521_01,Wind,732817.944444,732817.947917,732818.559028,1.0020,-0.23,-1.91,14.75,14.0,...,6.7,2.6,41975.6,17102.7,462.5,2.1,1.9,0.2,0.7,0.1
4,ICME_Wind_NASA_20070608_01,Wind,732835.239583,732835.239583,732836.218750,1.0040,-0.13,0.19,23.50,9.8,...,7.3,1.9,53830.2,25294.9,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,ICME_MAVEN_MOESTL_20170119_01,MAVEN,736348.243056,736348.625000,736349.770833,1.4262,-91.86,4.74,36.67,5.5,...,2.6,0.8,287437.6,242820.6,357.1,9.7,2.8,0.7,0.6,0.2
694,ICME_MAVEN_MOESTL_20170703_01,MAVEN,736513.930556,736514.875000,736516.194444,1.6225,-168.71,-2.80,54.33,7.8,...,4.8,1.4,369548.7,456549.3,378.7,6.1,2.6,1.2,0.6,0.3
695,ICME_MAVEN_MOESTL_20170722_01,MAVEN,736532.604167,736532.604167,736534.493056,1.6368,-177.51,-3.47,45.33,6.7,...,4.8,1.4,178292.6,220041.6,NaN,NaN,NaN,NaN,NaN,NaN
696,ICME_MAVEN_MOESTL_20170726_01,MAVEN,736536.187500,736536.375000,736537.506944,1.6396,-179.41,-3.60,31.67,21.4,...,2.4,0.8,1167580.7,1128618.7,666.2,0.0,0.5,0.0,0.4,0.0
